In [ ]:
import os
import torch
import Self_net_architecture
import tifffile as tif
import numpy as np

In [ ]:
def tensor_info(adrr) -> None:
    print(f'shape: {adrr.shape} \ndtype: {adrr.dtype} \nmax: {adrr.max()} \nmin: {adrr.min()} \nmean: {adrr.mean()} \nstd: {adrr.std()}')

# Model

In [ ]:
input_nc = 1
output_nc = 1
device=torch.device('cuda:0')

In [ ]:
deblur_net=Self_net_architecture.define_G(input_nc=input_nc, output_nc=output_nc, ngf=32, netG='care', device=device,use_dropout=False,norm='instance')
deblur_net.load_state_dict(torch.load('/home/ryuuyou/Project/self_net/data/visor/checkpoint/saved_models/deblur_net/50_250.pkl'))

# Data

In [ ]:
data_path = '/home/ryuuyou/Project/self_net/data/visor/raw_data/test'
data_name_list = os.listdir(data_path)
data_name_list.sort(key=lambda name:int(name.split('.')[0]))

result_save_path = '/home/ryuuyou/Project/self_net/data/visor/reconstruction_n'
if not os.path.exists(result_save_path):
    os.makedirs(result_save_path)

In [ ]:
print(data_name_list)

In [ ]:
deblur_net.eval()
res_stack=[]
for index, name in enumerate(data_name_list):
    img = tif.imread(os.path.join(data_path, name))
    img = np.expand_dims(np.expand_dims(img, axis=0), axis=0)
    img = img.astype(np.float32)
    img = (img - img.min())/(img.max() - img.min())
    img_tensor = torch.from_numpy(img).to(device=device).to(torch.float32)
    with torch.no_grad():
        res = deblur_net(img_tensor)
    # [x, z]
    res_npadrr = res.squeeze_(0).squeeze_(0).cpu().numpy()
    # res_npadrr = (res_npadrr - res_npadrr.min())/(res_npadrr.max() - res_npadrr.min())
    tif.imwrite(os.path.join(result_save_path, name), res_npadrr)
#     res_stack.append(res_npadrr)
# res_stack = np.asarray(res_stack, dtype=np.float32)
# res_stack = np.moveaxis(res_stack, -1, 0)
# tif.imwrite(os.path.join('/home/ryuuyou/Project/self_net/data/care_liver/reconstruction_stack.tif'), res_stack)